In [ ]:
import geopandas as gpd

In [ ]:
old_counties_geojson_df = gpd.read_file("counties_update.geojson")
old_counties_geojson_df.head(5)

In [ ]:
old_counties_geojson_df.columns.values

Extract useful columns

In [ ]:
counties_df = old_counties_geojson_df[['NAME','state_name','GEOID','population','geometry']]
counties_df.head(5)

Check if New York City and Kansas City Exists

In [ ]:
counties_df[counties_df['NAME'] == 'New York City']

In [ ]:
counties_df[counties_df['NAME'] == 'Kansas City']

Add Kansas City from external GeoJSON file

In [ ]:
kansas_city_geojson_df = gpd.read_file('kansas_city.geojson')
kansas_city_geojson_df.head(5)

In [ ]:
#Population Data get from WIKI: https://en.wikipedia.org/wiki/Kansas_City,_Missouri
#According to the U.S. Census Bureau, the city had an estimated population of 491,918 in 2018
kansas_geometry = kansas_city_geojson_df.geometry.unary_union
counties_df = counties_df.append(
    {
        'NAME':'Kansas City',
        'state_name':'Missouri',
        'GEOID':'00000',
        'population':491918,
        'geometry':kansas_geometry
    },ignore_index=True)

In [ ]:
counties_df[counties_df['NAME'] == 'Kansas City']

Add New York City by merging 5 counties

In [ ]:
new_york_city_df = counties_df[((counties_df['NAME'] == 'New York') 
            | (counties_df['NAME'] == 'Kings') 
            | (counties_df['NAME'] == 'Queens') 
            | (counties_df['NAME'] == 'Bronx')
            | (counties_df['NAME'] == 'Richmond'))
            & (counties_df['state_name'] == 'New York')]

In [ ]:
nyc_geometry = new_york_city_df.geometry.unary_union

In [ ]:
counties_df = counties_df.append(
    {
        'NAME':'New York City',
        'state_name':'New York',
        'GEOID':'00000',
        'population':new_york_city_df['population'].sum(),
        'geometry':nyc_geometry,
    },ignore_index=True)

In [ ]:
counties_df[counties_df['NAME'] == 'New York City']

In [ ]:
counties_df.to_file("counties_update_new.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

In [ ]:
_f# def get_shaply_geometry(county, state="New York"):
#     county_df = old_counties_geojson_df.loc[old_counties_geojson_df["NAME"]==county].loc[old_counties_geojson_df["state_name"]==state]
#     index = county_df.index[0]
#     geom = county_df['geometry'].values[0]
#     return geom, index

def get_shaply_geometry(county_state_pair_list):
    geom_list=[]
    index_list=[]
    for county_state_pair in county_state_pair_list:
        county = county_state_pair[0]
        state = county_state_pair[1]
        
        county_df = old_counties_geojson_df.loc[old_counties_geojson_df["NAME"]==county].loc[old_counties_geojson_df["state_name"]==state]
        index = county_df.index[0]
        index_list.append(index)
        geom = county_df['geometry'].values[0]
        geom_list.append(geom)
        print(county_state_pair)
        display(geom)
    return geom_list, index_list

In [ ]:
nyc_boroughs_list = [("New York","New York"), ("Kings","New York"), ("Queens","New York"), ("Bronx","New York"), ("Richmond","New York")]

borough_geom_list, borough_index_list = get_shaply_geometry(nyc_list)





In [ ]:
import shapely

In [ ]:
nyc_AllInOne_geom=shapely.ops.cascaded_union(borough_geom_list)

In [ ]:
#df.loc[df.shape[0]]
old_counties_geojson_subset_df=old_counties_geojson_df[["NAME", "state_name", "STATEFP", "COUNTYNS", "geometry", "population"]]

In [ ]:
#new_counties_df = old_counties_geojson_subset_df.loc[old_counties_geojson_subset_df.shape[0]]=["New York City", "Unknown", nyc, 1]

In [ ]:
gdf2 = gpd.geodataframe.GeoDataFrame([["New York City", "New York",None,None,nyc_AllInOne_geom, 1]], columns=["NAME", "state_name", "STATEFP", "COUNTYNS", "geometry", "population"])


In [ ]:
a=old_counties_geojson_subset_df.append(gdf2, ignore_index=True)

In [ ]:
a=a.drop(borough_index_list)

In [ ]:
a.to_file("counties_update_new.geojson", driver='GeoJSON')
print("done")